## CAPM & FRENCH FAMA ANALYSIS

# 1. Import libraries

In [6]:
!pip install pandas numpy matplotlib statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


# 2. Load Data

In [7]:
# swedish companies data loading
astrazeneca_data = pd.read_csv('Swedish_companies_251030_AH_AstraZeneca.csv')
axfood_data = pd.read_csv('Swedish_companies_251030_AH_Axfood.csv')
bonava_data = pd.read_csv('Swedish_companies_251030_AH_Bonava.csv')
government_bonds_data = pd.read_csv('Swedish_companies_251030_AH_Government_Bond_10_year.csv')
hufvudstaden_data = pd.read_csv('Swedish_companies_251030_AH_Hufvudstaden.csv')
sbb_data = pd.read_csv('Swedish_companies_251030_AH_SBB.csv')
sixrx_data = pd.read_csv('Swedish_companies_251030_AH_SIXRX.csv')

# US companies data loading
apple_data = pd.read_csv('US_companies_251030_AH_Apple.csv')
cocacola_data = pd.read_csv('US_companies_251030_AH_CocaCola.csv')
ff_data = pd.read_csv('US_companies_251030_AH_FF.csv')
microsoft_data = pd.read_csv('US_companies_251030_AH_Microsoft.csv')

# 3. Convert swedish risk free rate to weekly

In [8]:
rf_annual = government_bonds_data[['Annual_Yield']].copy()
rf_annual.columns = ['RF_Annual']
rf_weekly = (rf_annual / 100) * (7 / 365)
rf_weekly = rf_weekly.squeeze()  # Convert to Series

print(rf_weekly.head())
print(type(rf_weekly))


0    9.589041e-07
1    8.821918e-07
2    9.013699e-07
3    1.073973e-06
4    1.093151e-06
Name: RF_Annual, dtype: float64
<class 'pandas.core.series.Series'>


# 4. Compute weekly return for swedish stocks

In [9]:
def compute_returns(df):
    return df.pct_change().dropna()

returns_sweden = compute_returns(astrazeneca_data['Adjusted_Close_Price']).to_frame(name='AstraZeneca')
returns_sweden['Axfood'] = compute_returns(axfood_data['Adjusted_Close_Price'])
returns_sweden['Bonava'] = compute_returns(bonava_data['Adjusted_Close_Price'])
# returns_sweden['Government Bonds'] = compute_returns(government_bonds_data['Annual_Yield'])
returns_sweden['Hufvudstaden'] = compute_returns(hufvudstaden_data['Adjusted_Close_Price'])
returns_sweden['SBB'] = compute_returns(sbb_data['Adjusted_Close_Price'])
returns_sweden['SIXRX'] = compute_returns(sixrx_data['Close'])

# returns_us = compute_returns(apple_data['Adjusted_Close_Price']).to_frame(name='Apple')
# returns_us['CocaCola'] = compute_returns(cocacola_data['Adjusted_Close_Price'])
# # returns_us['FF'] = compute_returns(ff_data['Adjusted_Close_Price'])
# returns_us['Microsoft'] = compute_returns(microsoft_data['Adjusted_Close_Price'])  

print(returns_sweden.head())
# print(returns_us.head())

   AstraZeneca    Axfood    Bonava  Hufvudstaden       SBB     SIXRX
1     0.015680  0.090426  0.022831      0.015666  0.014652  0.019918
2     0.021424 -0.008014 -0.017857     -0.012495 -0.093863  0.015184
3    -0.020358 -0.008079  0.036364      0.012653  0.015936  0.018403
4    -0.123426  0.030099  0.035088      0.021973 -0.015686 -0.001097
5     0.034124 -0.002647 -0.118644     -0.004300  0.035857  0.005230


# 5. CAPM regression for each swedish stocks

In [10]:
# def run_capm(stock_ret, market_ret, rf):
#     y = stock_ret - rf
#     X = market_ret - rf
#     X = sm.add_constant(X)
#     model = sm.OLS(y, X).fit()
#     return model

# model = run_capm(returns_sweden['AstraZeneca'], returns_sweden['SIXRX'], rf_weekly)
# model.summary()

# def run_capm(stock_ret, market_ret, rf):
#     common_index = stock_ret.index.intersection(market_ret.index).intersection(rf.index)
#     y = stock_ret.loc[common_index] - rf.loc[common_index]
#     X = market_ret.loc[common_index] - rf.loc[common_index]
#     X = sm.add_constant(X)
#     model = sm.OLS(y, X).fit()
#     return model

# model = run_capm(returns_sweden['AstraZeneca'], returns_sweden['SIXRX'], rf_weekly)
# model.summary()

def run_capm(stock_ret, market_ret, rf):
    common_index = stock_ret.index.intersection(market_ret.index).intersection(rf.index)
    y = stock_ret.loc[common_index] - rf.loc[common_index]
    X = market_ret.loc[common_index] - rf.loc[common_index]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

model = run_capm(returns_sweden['AstraZeneca'], returns_sweden['SIXRX'], rf_weekly)
model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.747
Date:                Wed, 26 Nov 2025   Prob (F-statistic):              0.187
Time:                        12:51:34   Log-Likelihood:                 745.75
No. Observations:                 381   AIC:                            -1487.
Df Residuals:                     379   BIC:                            -1480.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0021      0.002     -1.213      0.226      -0.006       0.001
0              0.0868      0.066      1.322      0.187      -0.042       0.216
==============================================================================
Omnibus:                       37.351   Durbin-Watson:                   2.144
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              123.936
Skew:                           0.371   Prob(JB):                     1.22e-27
Kurtosis:                       5.694   Cond. No.                         37.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 6. Create Equally Weighted portfolio

In [11]:
def equal_weight_portfolio(df):
    return df.mean(axis=1)


portfolio = equal_weight_portfolio(returns_sweden[['AstraZeneca','Axfood','Bonava','Hufvudstaden','SIXRX','SBB']])
print(portfolio.head())

1    0.029862
2   -0.015937
3    0.009153
4   -0.008842
5   -0.008397
dtype: float64


# 7. Systematic VS Unsystematic error

In [12]:
def risk_decomposition(stock_ret, market_ret, beta):
    sigma_m = np.var(market_ret)
    systematic = (beta**2) * sigma_m
    unsystematic = np.var(stock_ret) - systematic
    return systematic, unsystematic

In [13]:
# Extract beta from the CAPM regression model
beta = model.params.iloc[1]  # The slope coefficient is beta

# Calculate systematic and unsystematic risk
systematic_risk, unsystematic_risk = risk_decomposition(
    returns_sweden['AstraZeneca'], 
    returns_sweden['SIXRX'], 
    beta
)

print(f"Beta: {beta:.6f}")
print(f"Systematic Risk: {systematic_risk:.6f}")
print(f"Unsystematic Risk: {unsystematic_risk:.6f}")
print(f"Total Risk: {systematic_risk + unsystematic_risk:.6f}")


Beta: 0.086772
Systematic Risk: 0.000005
Unsystematic Risk: 0.001168
Total Risk: 0.001173


## 8. Fama-French Three-Factor Regression (US Data)


In [ ]:
def run_ff3(stock_ret, rf, mkt, smb, hml):
    # Find common index across all series
    common_index = stock_ret.index.intersection(rf.index).intersection(mkt.index).intersection(smb.index).intersection(hml.index)
    
    # Align all data to common index
    stock_ret = stock_ret.loc[common_index]
    rf = rf.loc[common_index]
    mkt = mkt.loc[common_index]
    smb = smb.loc[common_index]
    hml = hml.loc[common_index]
    
    # Run regression
    y = stock_ret - rf
    X = pd.concat([mkt - rf, smb, hml], axis=1)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

# Run Fama-French 3-Factor model for all three US companies
print("=" * 80)
print("FAMA-FRENCH 3-FACTOR REGRESSION RESULTS")
print("=" * 80)

# Apple
print("\n" + "=" * 80)
print("APPLE (AAPL)")
print("=" * 80)
ff3_apple = run_ff3(apple_data['Adjusted_Close_Price'], ff_data['RF'], ff_data['Mkt-RF'], ff_data['SMB'], ff_data['HML'])
print(ff3_apple.summary())

# Microsoft
print("\n" + "=" * 80)
print("MICROSOFT (MSFT)")
print("=" * 80)
ff3_microsoft = run_ff3(microsoft_data['Adjusted_Close_Price'], ff_data['RF'], ff_data['Mkt-RF'], ff_data['SMB'], ff_data['HML'])
print(ff3_microsoft.summary())

# Coca-Cola
print("\n" + "=" * 80)
print("COCA-COLA (KO)")
print("=" * 80)
ff3_cocacola = run_ff3(cocacola_data['Adjusted_Close_Price'], ff_data['RF'], ff_data['Mkt-RF'], ff_data['SMB'], ff_data['HML'])
print(ff3_cocacola.summary())


In [26]:
# Summary comparison of all three companies
print("\n" + "=" * 100)
print("FAMA-FRENCH 3-FACTOR MODEL COMPARISON - ALL THREE COMPANIES")
print("=" * 100)

companies = {
    'Apple (AAPL)': ff3_apple,
    'Microsoft (MSFT)': ff3_microsoft,
    'Coca-Cola (KO)': ff3_cocacola
}

for company_name, model in companies.items():
    print(f"\n{company_name}:")
    print(f"  Alpha (Intercept):        {model.params.iloc[0]:8.6f}  (p-value: {model.pvalues.iloc[0]:.4f})")
    print(f"  Market Risk Premium Beta: {model.params.iloc[1]:8.6f}  (p-value: {model.pvalues.iloc[1]:.4f})")
    print(f"  SMB (Size) Beta:          {model.params.iloc[2]:8.6f}  (p-value: {model.pvalues.iloc[2]:.4f})")
    print(f"  HML (Value) Beta:         {model.params.iloc[3]:8.6f}  (p-value: {model.pvalues.iloc[3]:.4f})")
    print(f"  R-squared:                {model.rsquared:8.6f}")
    print(f"  Adj. R-squared:           {model.rsquared_adj:8.6f}")



FAMA-FRENCH 3-FACTOR MODEL COMPARISON - ALL THREE COMPANIES

Apple (AAPL):
  Alpha (Intercept):        95.018120  (p-value: 0.0000)
  Market Risk Premium Beta: 51.916250  (p-value: 0.7233)
  SMB (Size) Beta:          -147.142038  (p-value: 0.5278)
  HML (Value) Beta:         223.922774  (p-value: 0.1870)
  R-squared:                0.014982
  Adj. R-squared:           -0.006746

Microsoft (MSFT):
  Alpha (Intercept):        182.877907  (p-value: 0.0000)
  Market Risk Premium Beta: 177.566729  (p-value: 0.5297)
  SMB (Size) Beta:          -315.915933  (p-value: 0.4815)
  HML (Value) Beta:         302.579216  (p-value: 0.3538)
  R-squared:                0.010606
  Adj. R-squared:           -0.011219

Coca-Cola (KO):
  Alpha (Intercept):        43.840909  (p-value: 0.0000)
  Market Risk Premium Beta: 3.808920  (p-value: 0.8822)
  SMB (Size) Beta:          -37.639288  (p-value: 0.3575)
  HML (Value) Beta:         28.718644  (p-value: 0.3339)
  R-squared:                0.012207
  Adj. R-